# Single facet simulation - parametric study

TODO
 - Section 1 - costruct a displacement controlled model and show the load displacement curve for
   sliding
 - Section 2 - with the identified 

In [ ]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [ ]:
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [ ]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [ ]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_utils.api import Model

In [ ]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Calibrating monotonic behavior

# Kinematic hardening , $\gamma_\mathrm{T}$

In [ ]:
gamma_T_range = np.array(np.linspace(0,10000,10))

m_list = []

for gamma_T_ in gamma_T_range:
    print(gamma_T_)
    n_e = 1
    L_x = 1.0
    
    xd_inner = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_outer = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_inner.mesh.I[-1, :],
        J=xd_outer.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
    E_T=11111, gamma_T=gamma_T_, K_T=0, S_T=0.001, c_T=3, bartau=2, 
    E_N=66666, S_N=0.05, c_N = 1, m = 0.008, f_t=2, f_c=40, f_c0=20, eta=0.0 # 0.5
    )
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_inner, m1),
            (xd_outer, m2),
            (xd12, bond_m),
        ],
    )
    
    tf = TFCyclicNonsymmetricConstant(number_of_cycles=2, unloading_ratio=0.2)
    
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    outer_control_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    outer_control_dofs = outer_control_slice.dofs

    outer_control = [BCDof(var='u', dof=dof, value=0.4 ) 
                     for dof in outer_control_dofs ]

    outer_ring_fixed = BCSlice(slice=xd_outer.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value=0.0)
    inner_fixed = BCSlice(slice=xd_inner.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    bc1 = outer_control + [outer_ring_fixed, inner_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.005
    s.reset()
    s.run()
    m_list.append(m)
    


In [ ]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2), (ax3,  ax4)) = plt.subplots(2,2, tight_layout=True, figsize=(8,11))
fig.canvas.header_visible=False

for i in range(len(m_list)):
    m = m_list[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    outer_control_dofs = [bc.dof for bc in outer_control]
    F_l_t = np.sum(F_to[:, outer_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, outer_control_dofs], axis=-1)
    
    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }

    
    ax1.plot(U_r_t, -EpsSig_t['tau_pi_x'][:,0,0], label=str(gamma_T_range[i]))
    ax1.set_xlabel('s_x')
    ax1.set_ylabel('$\sigma_T$')
    ax1.legend()
    
    ax2.plot(U_r_t, -EpsSig_t['sig_pi'][:,0,0])
    ax2.set_xlabel('s_x')
    ax2.set_ylabel('$\sigma_N$')
    
    
    ax3.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
    ax3.set_xlabel('s_x')
    ax3.set_ylabel('$\omega_T$')
    
    
    ax4.plot(U_r_t, -EpsSig_t['omega_N'][:,0,0])
    ax4.set_xlabel('s_x')
    ax4.set_ylabel('$\omega_N$')
    




# Sliding interface damage strength, $S_\mathrm{T}$

In [ ]:
S_T_range = np.array(np.linspace(0.01,0.001,10))

m_list = []

for S_T_ in S_T_range:
    print(S_T_)
    n_e = 1
    L_x = 1.0
    
    xd_inner = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_outer = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_inner.mesh.I[-1, :],
        J=xd_outer.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
    E_T=11111, gamma_T=10000, K_T=0, S_T = S_T_, c_T=3, bartau=2, 
    E_N=66666, S_N=0.05, c_N = 1, m = 0.008, f_t=2, f_c=40, f_c0=20, eta=0.0 # 0.5
    )
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_inner, m1),
            (xd_outer, m2),
            (xd12, bond_m),
        ],
    )
    
    tf = TFCyclicNonsymmetricConstant(number_of_cycles=2, unloading_ratio=0.2)
    
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    outer_control_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    outer_control_dofs = outer_control_slice.dofs

    outer_control = [BCDof(var='u', dof=dof, value=0.4 ) 
                     for dof in outer_control_dofs ]

    outer_ring_fixed = BCSlice(slice=xd_outer.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value=0.0)
    inner_fixed = BCSlice(slice=xd_inner.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    bc1 = outer_control + [outer_ring_fixed, inner_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.001
    s.reset()
    s.run()
    m_list.append(m)
    


In [ ]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2), (ax3,  ax4)) = plt.subplots(2,2, tight_layout=True, figsize=(8,11))
fig.canvas.header_visible=False

for i in range(len(m_list)):
    m = m_list[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    outer_control_dofs = [bc.dof for bc in outer_control]
    F_l_t = np.sum(F_to[:, outer_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, outer_control_dofs], axis=-1)
    
    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }
    
    ax1.plot(U_r_t, -EpsSig_t['tau_pi_x'][:,0,0], label=str(S_T_range[i]))
    ax1.set_xlabel('s_x')
    ax1.set_ylabel('$\sigma_T$')
    ax1.legend()
    
    ax2.plot(U_r_t, -EpsSig_t['sig_pi'][:,0,0])
    ax2.set_xlabel('s_x')
    ax2.set_ylabel('$\sigma_N$')
    
    
    ax3.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
    ax3.set_xlabel('s_x')
    ax3.set_ylabel('$\omega_T$')
    
    
    ax4.plot(U_r_t, -EpsSig_t['omega_N'][:,0,0])
    ax4.set_xlabel('s_x')
    ax4.set_ylabel('$\omega_N$')
    




# Pressure sensitivity parameter, m 

In [ ]:
m_range = np.array(np.linspace(0.005,0.03,10))

m_list = []


for m_ in m_range:
    print(m_)
    n_e = 1
    L_x = 1.0
    
    xd_inner = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_outer = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_inner.mesh.I[-1, :],
        J=xd_outer.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
    E_T=11111, gamma_T=10000, K_T=0, S_T = 0.002, c_T=3, bartau=2, 
    E_N=66666, S_N=0.05, c_N = 1, m = m_, f_t=2, f_c= 40, f_c0=20, eta=0.0 # 0.5
    )
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_inner, m1),
            (xd_outer, m2),
            (xd12, bond_m),
        ],
    )
    
    tf = TFCyclicNonsymmetricConstant(number_of_cycles=2, unloading_ratio=0.2)
    
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    outer_control_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    outer_control_dofs = outer_control_slice.dofs

    outer_control = [BCDof(var='u', dof=dof, value=0.4 ) 
                     for dof in outer_control_dofs ]

    outer_ring_fixed = BCSlice(slice=xd_outer.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value=0.0)
    inner_fixed = BCSlice(slice=xd_inner.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    bc1 = outer_control + [outer_ring_fixed, inner_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.001
    s.reset()
    s.run()
    m_list.append(m)
    

In [ ]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2), (ax3,  ax4)) = plt.subplots(2,2, tight_layout=True, figsize=(8,11))
fig.canvas.header_visible=False

for i in range(len(m_list)):
    m = m_list[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    outer_control_dofs = [bc.dof for bc in outer_control]
    F_l_t = np.sum(F_to[:, outer_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, outer_control_dofs], axis=-1)
    
    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }
    
    ax1.plot(U_r_t, -EpsSig_t['tau_pi_x'][:,0,0], label=str(m_range[i]))
    ax1.set_xlabel('s_x')
    ax1.set_ylabel('$\sigma_T$')
    ax1.legend()
    
    ax2.plot(U_r_t, -EpsSig_t['sig_pi'][:,0,0])
    ax2.set_xlabel('s_x')
    ax2.set_ylabel('$\sigma_N$')
    
    
    ax3.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
    ax3.set_xlabel('s_x')
    ax3.set_ylabel('$\omega_T$')
    
    
    ax4.plot(U_r_t, -EpsSig_t['omega_N'][:,0,0])
    ax4.set_xlabel('s_x')
    ax4.set_ylabel('$\omega_N$')
    

# Calibrating fatigue behavior

# Sliding damage accumulation exponent, $c_\mathrm{T}$

In [ ]:
c_range = np.array(np.linspace(2.9,3,2))
m_list = []

for c_ in c_range:
    print(c_)
    n_e = 1
    L_x = 1.0
    
    xd_inner = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_outer = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_inner.mesh.I[-1, :],
        J=xd_outer.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
    E_T=11111, gamma_T=10000, K_T=0, S_T = 0.002, c_T=c_, bartau=2, 
    E_N=66666, S_N=0.05, c_N = 1, m = 0.02, f_t=2, f_c=40, f_c0=40, eta=0.0 # 0.5
    )
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_inner, m1),
            (xd_outer, m2),
            (xd12, bond_m),
        ],
    )
        
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    outer_control_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    outer_control_dofs = outer_control_slice.dofs

    outer_control = [BCDof(var='u', dof=dof, value=0.4 ) 
                     for dof in outer_control_dofs ]

    outer_ring_fixed = BCSlice(slice=xd_outer.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value=0.0)
    inner_fixed = BCSlice(slice=xd_inner.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    bc1 = outer_control + [outer_ring_fixed, inner_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.001
    s.reset()
    s.run()
    m_list.append(m)
    
    F_to = m.hist.F_t
    outer_control_dofs = [bc.dof for bc in outer_control]
    F_l_t = np.sum(F_to[:, outer_control_dofs], axis=-1)
    print(np.max(np.abs(F_l_t)))
    
    
    m = TStepBC(
        domains=[
            (xd_inner, m1),
            (xd_outer, m2),
            (xd12, bond_m),
        ],
    )
    
    
    tf = TFCyclicNonsymmetricConstant(number_of_cycles=10, unloading_ratio=0.05)
    
    
    F_max = 0.85 * np.max(np.abs(F_l_t)) / len(outer_control_dofs)
    outer_control = [BCDof(var='f', dof=dof, value=F_max, time_function=tf ) 
                         for dof in outer_control_dofs ]

    bc1 = outer_control + [outer_ring_fixed, inner_fixed]
    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 500
    s.tline.step = 0.001
    s.reset()
    s.run()
    m_list.append(m)

In [ ]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2), (ax3,  ax4)) = plt.subplots(2,2, tight_layout=True, figsize=(8,11))
fig.canvas.header_visible=False

for i in range(len(m_list)):
    
    m = m_list[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    outer_control_dofs = [bc.dof for bc in outer_control]
    F_l_t = np.sum(F_to[:, outer_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, outer_control_dofs], axis=-1)

    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }
    
    if i % 2 == 0:
        
        ax1.plot(U_r_t, -EpsSig_t['tau_pi_x'][:,0,0])
        ax1.plot(U_r_t, -EpsSig_t['sig_pi'][:,0,0])
        ax1.set_xlabel('s_x')
        ax1.set_ylabel('$\sigma_T$')
        #ax1.legend()


        ax2.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
        ax2.set_xlabel('s_x')
        ax2.set_ylabel('$\omega_T$')
    else:
        
        
        ax3.plot(U_r_t, -EpsSig_t['tau_pi_x'][:,0,0])
        ax3.plot(U_r_t, -EpsSig_t['sig_pi'][:,0,0])
        ax3.set_xlabel('s_x')
        ax3.set_ylabel('$\sigma_T$')
        #ax3.legend()


        ax4.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
        ax4.set_xlabel('s_x')
        ax4.set_ylabel('$\omega_T$')

        
    




In [ ]:
if False:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()